# Test SAT noise in spectra version 3.1.1

## First generate spectra using `so_noise_models`

In [ ]:
from so_models_v3 import SO_Noise_Calculator_Public_v3_1_1 as so_models

import matplotlib
matplotlib.rc('text', usetex=False)
fontProperties = {
                  'weight' : 'normal', 'size' : 16}
import matplotlib.pyplot as plt

####################################################################
####################################################################
##                   demonstration of the code
####################################################################

mode=1 # baseline
fsky=1
ellmax=1e4
el=None

dset_label = 'LAT\\_V3.1'
tel = so_models.SOSatV3point1(mode, el=el)

print(dset_label)
bands = tel.get_bands()
print("band centers: ", tel.get_bands(), "[GHz]")
print("beam sizes: "  , tel.get_beams(), "[arcmin]")
N_bands = len(bands)

ell, _,N_ell_LA_P = tel.get_noise_curves(
    fsky, ellmax, 1, full_covar=False, deconv_beam=False, rolloff_ell=50)

WN_levels = tel.get_white_noise(fsky)**.5

N_ell_LA_T  = N_ell_LA_P / 2

print("white noise levels: "  , WN_levels, "[uK-arcmin]")

In [ ]:
%matplotlib inline

In [ ]:
import healpy as hp

In [ ]:
nside = 256
lmax = 3 * nside -1

## Simulate maps with `mapsims`

In [ ]:
from mapsims import SONoiseSimulator
sim = SONoiseSimulator(nside=256,
                       rolloff_ell=50, full_covariance=False, homogeneous=True,
                       apply_beam_correction=False)

In [ ]:
ell_mapsims, cl_T, cl_P = sim.get_fullsky_noise_spectra(tube="ST3")

In [ ]:
tube = "ST3"

In [ ]:
m = sim.simulate(tube=tube, seed=124)

In [ ]:
hp.mollview(m[0][0][0], min=-100, max=100)

## Take spectra and cross-spectra of the maps and compare them

In [ ]:
cross_noise_cl = hp.anafast(m[0][0], m[1][0],
                            lmax=lmax, use_pixel_weights=True)

In [ ]:
cross_noise_cl.shape

In [ ]:
bands

In [ ]:
cross_noise_cl.std()

In [ ]:
# No cross-correlation

assert cross_noise_cl.std() < 1e-4            

In [ ]:
noise_cl_27 = hp.anafast(m[0][0],
                            lmax=lmax, use_pixel_weights=True)

In [ ]:
plt.figure()
for i in range(1):
    plt.loglog(ell,N_ell_LA_T[i], label='%i GHz (%s)' % (bands[i], dset_label),
               ls='-', lw=2.)

# include correlated atmospheric noise across frequencies
#for _c,(i,j) in enumerate(corr_pairs[:1]):
#    plt.loglog(ell, N_ell_LA_T_full[i,j],
#               label=r'$%i \times %i$ GHz atm.' % (bands[i],bands[j]),
#               lw=1.5)

#plt.loglog(cross_noise_cl[0], lw=1, label="Cross spectra from maps")
plt.loglog(noise_cl_27[0], lw=1, label="27 GHz spectrum from maps")

plt.title(r"$N(\ell$) Temperature", fontsize=18)
plt.ylabel(r"$N(\ell$) [$\mu$K${}^2$]", fontsize=16)
plt.xlabel(r"$\ell$", fontsize=16)
plt.ylim(1e-10,1e-2)
plt.xlim(0,10000)
plt.legend(loc='lower left', ncol=2, fontsize=8)
plt.grid()

In [ ]:
import numpy as np

In [ ]:
np.testing.assert_allclose(noise_cl_27[0][80:300],
                           N_ell_LA_T[0][80+2:300+2],
                           rtol=.3)

## Polarization

In [ ]:
plt.figure(figsize=(10,5))
#for i in range(2):
#    plt.loglog(ell,N_ell_LA_P[i], label='%i GHz (%s)' % (bands[i], dset_label),
#               color=colors[i], ls='-', lw=2.)

# include correlated atmospheric noise across frequencies
plt.loglog(ell, N_ell_LA_P[0],
               label="27 GHz spectrum from so_noise",
               lw=2)

plt.loglog(noise_cl_27[1], lw=1, label="EE spectrum from map")
plt.loglog(noise_cl_27[2], lw=1, label="BB spectrum from map")

plt.title(r"$N(\ell$) Polarization", fontsize=18)
plt.ylabel(r"$N(\ell$) [$\mu$K${}^2$]", fontsize=16)
plt.xlabel(r"$\ell$", fontsize=16)
plt.ylim(1e-7, 100)
plt.xlim(0,1000)
plt.legend(loc='lower left', ncol=2, fontsize=8)
plt.grid()

In [ ]:
np.testing.assert_allclose(noise_cl_27[1][80:300],
                           N_ell_LA_P[0][80+2:300+2],
                           rtol=.5)

In [ ]:
np.testing.assert_allclose(noise_cl_27[2][80:300],
                           N_ell_LA_P[0][80+2:300+2],
                           rtol=.5)

## Test that the seed is working

result should be deterministic

In [ ]:
expected_cross_noise_cl_2 = np.array([0.00108629, 0.00094161, 0.00108058, 0.00098768, 0.00107019,
       0.00106682, 0.00095412, 0.00102593, 0.00096112, 0.00108508])

In [ ]:
np.testing.assert_allclose(expected_cross_noise_cl_2, noise_cl_27[2][300:310], rtol=1e-3)